<a href="https://colab.research.google.com/github/cheacheaza0066/5.30/blob/master/skin_de.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np #ใช้คำนวน
import cv2 #จัดการรูปภาพ
import tensorflow as tf
from os import listdir #จัดการไฟล
from os.path import isfile, join 
from tqdm import tqdm #Monitor Process
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten , Conv2D, MaxPool2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/Testthai-20000file.zip

!unzip /content/drive/MyDrive/test.zip


In [ ]:
width = 128 #ขนาดรูปภาพ
num_classes = 13 #จำนวน Class
trainpath = 'train/'
testpath = 'test/'
trainImg = [trainpath+f for f in listdir(trainpath)]
testImg = [testpath+f for f in listdir(testpath)]

In [ ]:
trainImg , testImg

In [ ]:
#สร้าง Function สำหรับอ่านรูปภาพ เเละ สร้าง labels 

def img2data(path):
  rawImgs = []
  labels = []
  for imagePath in (path):
      for item in tqdm(listdir(imagePath)):
          file = join(imagePath, item)
          if file[-1] =='g':
            img = cv2.imread(file , cv2.COLOR_BGR2RGB)
            img = cv2.resize(img ,(width,width))
            rawImgs.append(img)
            l = imagePath.split('/')[1]
            if l == 'สิว':
              labels.append([1,0,0,0,0,0,0,0,0,0,0,0,0])         
            elif l == 'กลาก':
              labels.append([0,1,0,0,0,0,0,0,0,0,0,0,0])         
            elif l == 'ด่างขาว':
              labels.append([0,0,1,0,0,0,0,0,0,0,0,0,0])
            elif l == 'ตุ่มพองน้ำใส':
              labels.append([0,0,0,1,0,0,0,0,0,0,0,0,0])
            elif l == 'ผิวหนังอักเสบจากต่อมไขมัน':
              labels.append([0,0,0,0,1,0,0,0,0,0,0,0,0])
            elif l == 'ลมพิษ':
              labels.append([0,0,0,0,0,1,0,0,0,0,0,0,0])
            elif l == 'หูดข้าวสุก':
              labels.append([0,0,0,0,0,0,1,0,0,0,0,0,0])
            elif l == 'เกลื้อน':
              labels.append([0,0,0,0,0,0,0,1,0,0,0,0,0])
            elif l == 'โรคผื่นภูมิแพ้ผิวหนังหรือผิวหนังอักเสบ':
              labels.append([0,0,0,0,0,0,0,0,1,0,0,0,0])
            elif l == 'โรคสะเก็ดเงิน':
              labels.append([0,0,0,0,0,0,0,0,0,1,0,0,0])
            elif l == 'โรคหลอดเลือดอักเสบ':
              labels.append([0,0,0,0,0,0,0,0,0,0,1,0,0])
            elif l == 'โรคหิด':
              labels.append([0,0,0,0,0,0,0,0,0,0,0,1,0])
            elif l == 'โรคเริม':
              labels.append([0,0,0,0,0,0,0,0,0,0,0,0,1])
  return rawImgs, labels # retuen ค่า pixleรูปภาพ เเละ labels

In [ ]:
#สร้างตัวเเปลมารับค่าจาก fucntion img2data
# x_train เก็บค่ารูปภาพ 
# y_train เก็บ labels

x_train, y_train = img2data(trainImg)
x_test, y_test = img2data(testImg)

100%|██████████| 36/36 [00:00<00:00, 193.41it/s]


In [ ]:
# เเปลงเป็น numpy array เเล้วเปลียนข้อมูลเป็น float16
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')

# ต้องต้องแปลงเป็นตัวเลข
# แบบ float16 หรือตัวเลขที่มีจุดทศนิยม 3 ตำ แหน่ง เนื่องจากเมื่อใช้float32 หน่วยความจำ หลักจะไม่เพียงพอ
# ไม่สามารถดำ เนินการได
x_train = x_train.astype('float16') 
x_test = x_test.astype('float16')
x_train /= 255 #x_train เป็นค่า rgb 0-255,นำข้อมูลเข้า keras ไม่เกิน 1 ต้องหาร 255
x_test /= 255

In [ ]:
# x_train.shape,y_train.shape,x_test.shape, y_test.shape

((19332, 128, 128, 3), (19332, 13), (536, 128, 128, 3), (536, 13))

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(128,128,3),
                                                   include_top=False,
                                                   weights='imagenet')
base_model.trainable = False
num_classes = 13

model = Sequential([
        base_model,
        Conv2D(128, 3, activation='relu'),
        MaxPool2D(pool_size=(2,2 )),
        Dense(16),
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])



In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
              loss='categorical_crossentropy', 
              metrics= ['accuracy'])
batch_size = 128 #กำหนดข้อมูลที่นำเข้า NN Network
epochs = 100

In [ ]:
history = model.fit(x_train, y_train ,batch_size=batch_size, epochs=epochs ,validation_data=(x_test, y_test))


Epoch 1/100
152/152 [==============================] - 25s 76ms/step - loss: 1.0304 - accuracy: 0.6986 - val_loss: 1.7896 - val_accuracy: 0.4534
Epoch 2/100
152/152 [==============================] - 10s 64ms/step - loss: 0.1963 - accuracy: 0.9648 - val_loss: 1.8130 - val_accuracy: 0.4646
Epoch 3/100
152/152 [==============================] - 10s 64ms/step - loss: 0.0635 - accuracy: 0.9950 - val_loss: 1.8351 - val_accuracy: 0.5093
Epoch 4/100
152/152 [==============================] - 9s 62ms/step - loss: 0.0299 - accuracy: 0.9988 - val_loss: 1.8487 - val_accuracy: 0.5168
Epoch 5/100
152/152 [==============================] - 10s 64ms/step - loss: 0.0187 - accuracy: 0.9993 - val_loss: 1.9182 - val_accuracy: 0.5093
Epoch 6/100
152/152 [==============================] - 10s 64ms/step - loss: 0.0116 - accuracy: 0.9999 - val_loss: 1.9855 - val_accuracy: 0.5131
Epoch 7/100
152/152 [==============================] - 10s 63ms/step - loss: 0.0084 - accuracy: 0.9999 - val_loss: 2.0069 - val_acc

In [ ]:
testpath = 'test/'
testImg = [testpath+f for f in listdir(testpath) if listdir(join(testpath, f))]
rimg = []
for imagePath in (testImg):
    for item in (os.listdir(imagePath)):
        file = os.path.join(imagePath, item)
        if item.split('.')[0] != "":
           
          img = cv2.imread(file , cv2.COLOR_BGR2RGB)
          ori = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
          img = cv2.resize(img ,(width,width))
          rimg = np.array(img)
          rimg = rimg.astype('float32')
          rimg /= 255
          rimg = np.reshape(rimg ,(1,128,128,3))
          predict = model.predict(rimg)
          label = ['สิว','กลาก','ด่างขาว','ตุ่มพองน้ำใส','ผิวหนังอักเสบจากต่อมไขมัน','ลมพิษ','หูดข้าวสุก','เกลื้อน','โรคผื่นภูมิแพ้ผิวหนังหรือผิวหนังอักเสบ','โรคสะเก็ดเงิน','โรคหลอดเลือดอักเสบ','โรคหิด','โรคเริม']
          # label = ['Acne','Ringworm']
          result = label[np.argmax(predict)]
          print(predict)
          print('real:'+str(item))
          print('predict:'+str(result))
          plt.imshow(ori)
          plt.show()



        

In [ ]:
model.save("model.h5")

In [ ]:
# Converting a SavedModel to a TensorFlow Lite model. 
saved_model_dir = r'/content/model.h5' 
loaded_model = tf.keras.models.load_model(saved_model_dir)
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
tflite_model = converter.convert()
open("my_model_converted_model.tflite", "wb").write(tflite_model)


14779488